## RSA Cipher



### Key generation

* Choose two primes $p$ and $q$

* Compute $n=pq$

* Compute the least common multiple of $p-1$ and $q-1$, and call it $\lambda(n)=lcm(p-1,q-1)$

* Choose an integer $e$ coprime to $\lambda(n)$

* Compute the inverse $d$ of $e$ modulo $\lambda(n)$ 

Now, say that Alice wants to receive from Bob a message. 

* $(n,e)$ is a public key, which Alice sends to Bob through a reliable channel. 
* $(n,d)$ is a private key, which Alice keeps for herself. 




See [Wikipedia](https://en.wikipedia.org/wiki/RSA_%28cryptosystem%29) for more details. See also [Khan Academy](https://www.khanacademy.org/computing/computer-science/cryptography#modern-crypt) for a great video introduction. 


### Encryption and Decryption 

Bob translates his message M to an integer $m$, and then converts it to ciphertext using 

$$ c = m^e \pmod{n} $$

When Alices receives the value of 'c', she decodes it using 

$$ m = c^d \pmod{n}$$ 

which recovers $m$. 

### Examples

* Choose $p=61$, $q=53$ which are two prime numbers
* Compute $n=pq = 3233$, 
* Compute the least common multiple, 

$$ \lambda (3233) = lcm (60, 52) = 780 $$

* Choose between $1<e<780$ this is coprime to 780. Let $e=17$. 
* Compute $d$, the modular multiplicative inverse of $ 17 \pmod {780}$.  
   
   - $780=2^2*3*5*13$ 
   - $\phi(780) = (2-1)^2(3-1)(5-1)(13-1) = 96$
   - $ d= 17^{96-1} \pmod{780} = 413$

Now the public/encryption key is $(n=3233, e=17)$, the private/decryption key is $(n=3233, d=413)$. 

In this case, $p$ and $q$ are small, it is very easy to guess their values from $n$. In reality, large prime numbers are used such that the decomposition of $n$ to prime numbers is difficult. 

Now let's try the encryption and decryption 

Bob wants to send a message "HI", e.g., H=7, I=8. He first encodes it with the key he got from Alice $(n=3233, e=17)$,

$$ c1 = 7^{17} \pmod{3233} = 2369 $$
$$ c2 = 8^{17} \pmod{3233} = 2041 $$

He sends "2369, 2041" to Alice. 

Alice now uses her private key $(n=3233, d=413)$ to decode it, 

$$ m1 = 2369^{413} \pmod{3233} = 7 $$
$$ m2 = 2041^{413} \pmod{3233} = 8 $$

and she gets "7 8". With the alphabetic table, she understands the message is "HI". 

The power mod $a^b \pmod {m}$ may be computed by calculators via the binary decomposition of the exponent $b$, see [Modular Arithemtic/Compute power modulo with a scientific calculator](../Modular/Readme.ipynb) for detailed steps.



### Codebusters Examples

There are not many examples available since RSA only shows up in Div C National/State. However, in [toebes.com](https://toebes.com/codebusters/RSAEncrypt.html), there are some different types of examples and solutions.

The most difficult part is to calculate the modular multiplicative inverse for large numbers. See [Modular Arithmetic](../Modular/Readme.ipynb) for more details. With Euler's totient function, a calculator cannot handle the power function $x^y$ for large numbers. We need to use the extended Euclidean algorithm. 

**Example** (from toebes.com): Special Agent, Grace, has the following RSA public key:   n = 109691   e = 54395 Unfortunately for her, A quantum computer has successfully factored her n   109691 = 229 * 479. Compute the value of her private key.

Since $p=229$, $q=479$, the first step is easy, to compute $\lambda(n) = lcm(229-1, 479-1)$. 

To compute lcm (least common multiplier), we factorize them into prime numbers. To check whether a number is  a prime number, we can divide it by common prime numbers up to its square root. 

```
p-1 = 228 = 2*114 =2*2*57 =2*2*3*19 
q-1 = 478 = 2*239     239 is prime ?? 239/3=, 239/7 = , 239/11 =,  239/13=, till sqrt(239)=15.46
```
therefore $\lambda(n) = 2*114*239 = 54492$

To get $d$, we need to solve $d*e \pmod{\lambda(n)} = 1$, or $d*54395\pmod{54492}=1$. 

Following Extended Euclidean algorithm, we continue to divide the remainders into a quotient ($q_i$)-remainder ($r_i$) form, 

$$r_{i-2} = q_{i} * r_{i-1} + r_{i}$$  

with the first two remainders set to $\lambda(n)$ and $e$, the prefactors to $e$ are computed as 
$d_i = d_{i-2} - q_i * d_{i-1}$, with first two values set as 0, and 1. Iterative until the remainder is 1.  

```
# step0                  54492: d0 = 0
# step1                  54397: d1 = 1  
# step2  54492 = 1*54395 + 97 : d2 = d0-1*d1 = -1 
# step3  54397 = 560*97  + 75 : d3 = d1-560*d2 = 561
# step4     97 =   1*75  + 22 : d4 = d2-1*d3 = -562
# step5     75 =   3*22  + 9  : d5 = 561-3*(-562)=2247
# step6     22 =   2*9   + 4  : d6 = -562-2*2247 = −5056
# step7     9  =   2*4   + 1  : d7 = 2247-2*(-5056) = 12359
```

And we get $d=12359$. 


### Python routines 

See [RSACipher.py](RSACipher.py) for native implementations of all integer operations. For faster processing, especially for large numbers, use [RSACipherGMP.py](RSACipherGMP.py) which uses [GNU MP Library](https://gmplib.org/manual/) and [gmpy2](https://gmpy2.readthedocs.io) interface.

#### Key generation / Encrypt / Decrypt

In [1]:
# import my RSA python module
import RSACipher as rsa

# generate public and private key pairs
# bits limit the range of prime numbers, 2^bits[0], 2^bits[1]
# or (2^2, 2^8) here
public_key, private_key = rsa.generate_key(bits=(2, 8), debug=True)


# get n from the public_key
e, n = public_key 

# message to be sent 
plain = 1603 % n 
# encrypt by Bob with the Alice's public_key
cipher = rsa.encrypt(plain, public_key)
# Alice uses her private key to decrypt
decipher = rsa.decrypt(cipher, private_key)

print("plain :", plain, "cipher: ", cipher, "decrypted: ", decipher)

RSA Key Generator
p=  61  q=  109
lambda(n) = 540
Public/Encryption Key  (e, n): (253, 6649)
Private/Decryption Key (d, n): (397, 6649)
plain : 1603 cipher:  3020 decrypted:  1603


#### Crack a RSA key

One way to crack RSA key is to factorize $n$ into $p$ and $q$, and $e$ can be easily found out. For a small $n$, it is possible, for example, by [Fermat's factorization method](https://en.wikipedia.org/wiki/Fermat%27s_factorization_method). 

In [2]:
p, q = rsa.fermat_factors(13631579)
print(p,q)

13 1048583


Of course. Larger numbers take longer time to factorize, which can be observed by the test below.   
**NOTE: THE FOLLOWING CODE MAY TAKE A LONG TIME TO FINISH. IF YOU DONT WANT TO WAIT, USE THE MENU Kernel->Restart TO STOP IT**

In [3]:
# define a timer function
def timer(func, *args): 
    import time
    start_time = time.time()
    func(*args)
    end_time = time.time()
    return end_time-start_time

### USE GMP version instead for large integers
import RSACipherGMP as gmp

for bits in range(6,30,2):
    prime = gmp.next_prime(2**bits)
    n = 13*prime 
    duration = timer(gmp.fermat_factors, n)
    print(f"time to factorize {n}=13x{prime} is {duration}")

time to factorize 871=13x67 is 4.696846008300781e-05
time to factorize 3341=13x257 is 2.4080276489257812e-05
time to factorize 13403=13x1031 is 0.00010585784912109375
time to factorize 53287=13x4099 is 0.0006520748138427734
time to factorize 213343=13x16411 is 0.0021719932556152344
time to factorize 851981=13x65537 is 0.008655309677124023
time to factorize 3407911=13x262147 is 0.034008026123046875
time to factorize 13631579=13x1048583 is 0.1318073272705078
time to factorize 54526147=13x4194319 is 0.5204660892486572
time to factorize 218104367=13x16777259 is 2.247535228729248
time to factorize 872415427=13x67108879 is 8.736737728118896
time to factorize 3489660967=13x268435459 is 35.254956007003784


#### Use Python to solve Codebusters problems

Example: Special Agent, Grace, has the following RSA public key:   n = 109691   e = 54395 Unfortunately for her, A quantum computer has successfully factored her n   109691 = 229 * 479. Compute the value of her private key.

In [4]:
# import the python package 
import RSACipher as rsa

# assign p, q 
p = 229
q = 479 

# find lambda(n) = lcm(p-1, q-1)
lambda_n = rsa.lcm(p-1, q-1) 
print(f"lambda(n) is {lambda_n}")

# e is given 
e = 54395 

# d is the modular mulplicative inverse of e mod(lambda_n)
d = rsa.invert(e, lambda_n)
print(f"The private key d is {d}")

lambda(n) is 54492
The private key d is 12359
